# Synthetic Data Generation (Encoding/Decoding) using Auto Encoders
#### Contractive autoencoders
Author: Gabriel Rodrigues Palma

Purpose: Encode and decode 5G data. Also, generate new dasets based on the autoencoders methods.

Sourcers: Contractive autoencoders code based on the paper: 

$\textbf{Contractive Auto-Encoders: Explicit Invariance During Feature Extraction}$ 

$\textit{Salah Rifai, Pascal Vincent, Xavier Muller, Xavier Glorot, and Yoshua Bengio. Contractive auto-encoders: Explicit invariance during feature extraction. In Proceedings of the 28th International Conference on International Conference on Machine Learning, ICML’11, pages 833–840, USA, 2011. Omnipress.}$

# Packages

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keract import get_activations, display_activations

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Main functions

In [2]:
def transformCategoricalToBinary(data):
    '''Function created to transform categorical data to binary arrays'''
    reshapedData = np.array(data).reshape(len(data), -1)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(reshapedData)    
    return(onehot_encoded)

def transformBinaryToCategorical(data, query):
    '''Function created to transform binary arrays data to categorical'''
    reshapedData = np.array(data).reshape(len(data), -1)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(reshapedData)   
    query = query

    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(list(np.unique(reshapedData)))
    inverted = label_encoder.inverse_transform(list(np.argmax(query, axis=1))) 
    return(inverted)

def transformScaledToNonscaled(data, query):
    '''Function to obtain original data from scaled one'''
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(data) 
    if query.ndim ==1:
        query = query.reshape(1, len(query))
    else:
        query = query.reshape(len(query), -1)
    result = scaler.inverse_transform(query)

    return(result)

def getOriginalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_numerical_data = decoded_data[:, 0:8]
    decoded_categorical_data = np.round(decoded_data[:, 8:42])
    numerical_data = transformScaledToNonscaled(data = original_data, 
                                                query = decoded_numerical_data)
    categorical_data = transformBinaryToCategorical(data = categorical_data, 
                                                    query = decoded_categorical_data)    
    result = np.hstack((numerical_data, categorical_data.reshape(len(categorical_data), 1)))

    return(result)

def getOriginalNumericalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_numerical_data = decoded_data   
    numerical_data = transformScaledToNonscaled(data = original_data, 
                                                query = decoded_numerical_data)
    result = numerical_data

    return(result)
def getOriginalCategoricalData(original_data, categorical_data, decoded_data):
    '''Function to obtain original data from decoded data'''
    decoded_categorical_data = np.round(decoded_data)
    categorical_data = transformBinaryToCategorical(data = categorical_data, 
                                                    query = decoded_categorical_data)  
    result = numerical_data

    return(result)

# Loading data

In [3]:
original_data = pd.read_csv('../../../Data/5G-production-dataset/Download/Static/B_2019.12.16_13.40.04.csv')
encoded_text_data = transformCategoricalToBinary(original_data['PINGMAX'])
original_data.describe()

,Longitude,Latitude,Speed,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID
count,2.143000e+03,2.143000e+03,2143.0,2143.0,2143.000000,2143.000000,2143.000000,2143.00000,2143.000000,2143.000000,2143.0
mean,-8.394628e+00,5.188614e+01,0.0,11.0,-95.999067,-13.856276,3.717219,10.65329,50382.109193,141.879608,10805003.0
std,1.221694e-07,9.162706e-08,0.0,0.0,1.274617,2.239170,3.260739,1.91965,65620.519457,122.211380,0.0
min,-8.394628e+00,5.188614e+01,0.0,11.0,-104.000000,-19.000000,-7.000000,6.00000,0.000000,0.000000,10805003.0
25%,-8.394628e+00,5.188614e+01,0.0,11.0,-96.000000,-16.000000,2.000000,10.00000,3229.500000,44.000000,10805003.0
50%,-8.394628e+00,5.188614e+01,0.0,11.0,-96.000000,-14.000000,4.000000,11.00000,23853.000000,128.000000,10805003.0
75%,-8.394628e+00,5.188614e+01,0.0,11.0,-95.000000,-12.000000,6.000000,11.00000,61175.000000,215.000000,10805003.0
max,-8.394624e+00,5.188614e+01,0.0,11.0,-93.000000,-9.000000,15.000000,15.00000,302694.000000,1215.000000,10805003.0


In [4]:
encoded_text_data_test = transformCategoricalToBinary(np.unique(original_data['PINGMAX']))

In [5]:
selected_data = original_data[['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate','UL_bitrate','RAWCELLID']]
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(selected_data)
data_scaled = scaler.transform(selected_data)

In [6]:
print(data_scaled.shape)
print(encoded_text_data_test.shape)

(2143, 8)
(34, 34)


In [7]:
data = np.hstack((data_scaled, encoded_text_data))
X_train, X_test, y_train, y_test = train_test_split(data, data, test_size=0.3, random_state=42)
X_train[:, 8:42].shape

(1500, 34)

# Conditional Generative Adversarial Networks (GANs)

## Working with numerical data

In [10]:
# Create the discriminator.
discriminator = keras.Sequential(name="discriminator")
discriminator.add(Input(shape=(8, )))
# Create the generator.
generator = keras.Sequential(name="generator")
generator

In [14]:
lam = 0.01
input_tensor1 = Input(shape=(8, ))
hidden_layer_11 = Dense(7)(input_tensor1)
hidden_layer_21 = Dense(5)(hidden_layer_11)
hidden_layer_31 = Dense(2)(hidden_layer_21)
hidden_layer_41 = Dense(5)(hidden_layer_31)

input_tensor2 = Input(shape=(34, ))
hidden_layer_12 = Dense(30)(input_tensor2)
hidden_layer_22 = Dense(20)(hidden_layer_12)
hidden_layer_32 = Dense(10)(hidden_layer_22)
hidden_layer_42 = Dense(2)(hidden_layer_32)
hidden_layer_52 = Dense(10)(hidden_layer_42)
hidden_layer_62 = Dense(20)(hidden_layer_52)

output_numeric = Dense(8, activation = 'sigmoid')(hidden_layer_41)
output_categorical = Dense(34, activation = 'sigmoid', activity_regularizer=L1(lam))(hidden_layer_62)

In [15]:
autoencoder = Model([input_tensor1, input_tensor2], [output_numeric, output_categorical])
autoencoder.compile(optimizer = 'adadelta', 
                    loss = ['mse', 'binary_crossentropy'], 
                    metrics = ['mse'])

In [ ]:
autoencoder.fit([X_train[:,0:8], X_train[:,8:42]], [X_train[:,0:8], X_train[:,8:42]], epochs=1000)

Epoch 1/1000


2022-07-19 13:14:10.069352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 2s 12ms/step - loss: 0.9747 - dense_31_loss: 0.1351 - dense_32_loss: 0.6729 - dense_31_mse: 0.1351 - dense_32_mse: 0.2399
Epoch 2/1000
47/47 [==============================] - 0s 10ms/step - loss: 0.9740 - dense_31_loss: 0.1351 - dense_32_loss: 0.6723 - dense_31_mse: 0.1351 - dense_32_mse: 0.2396
Epoch 3/1000
47/47 [==============================] - 0s 10ms/step - loss: 0.9732 - dense_31_loss: 0.1350 - dense_32_loss: 0.6718 - dense_31_mse: 0.1350 - dense_32_mse: 0.2393
Epoch 4/1000
47/47 [==============================] - 0s 10ms/step - loss: 0.9725 - dense_31_loss: 0.1349 - dense_32_loss: 0.6712 - dense_31_mse: 0.1349 - dense_32_mse: 0.2390
Epoch 5/1000
47/47 [==============================] - 0s 10ms/step - loss: 0.9717 - dense_31_loss: 0.1348 - dense_32_loss: 0.6706 - dense_31_mse: 0.1348 - dense_32_mse: 0.2387
Epoch 6/1000
47/47 [==============================] - 0s 10ms/step - loss: 0.9708 - dense_31_loss: 0.1347 - dense_32_loss: 0.6699 - d

# Obtaining encoded data

In [38]:
autoencoder.layers[-2]

## Obtaining numerical encoded and decoded data

In [80]:
# Build your encoder by using the first layer of your autoencoder
encoder = Sequential()
encoder.add(autoencoder.layers[0])

# Encode the noisy images and show the encodings for your favorite number [0-9]
encoded_data = encoder.predict(X_train[:,0:8])

#show_encodings(encodings, number = 1)
# Predict on the noisy images with your autoencoder
decoded_data = autoencoder.predict([X_train[:,0:8], X_train[:,8:42]])

47/47 [==============================] - 0s 2ms/step


2022-07-12 14:44:28.134341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


# Showing autoencoder results

## Original data

In [79]:
original_csv = getOriginalData(original_data = selected_data, 
                              categorical_data = original_data['PINGMAX'], 
                              decoded_data = X_train)
original_csv = pd.DataFrame(original_csv, 
             columns = ['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate',
                        'UL_bitrate','RAWCELLID', 'PINGMAX'])
original_csv

,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID,PINGMAX
0,11.0,-95.99999999999999,-17.0,2.9999999999999996,8.0,61756.0,256.0,10805003.0,-
1,11.0,-97.0,-16.0,4.999999999999999,8.0,25025.0,145.0,10805003.0,-
2,11.0,-95.99999999999999,-12.000000000000002,7.0,10.000000000000002,6614.999999999999,27.999999999999996,10805003.0,79.0
3,11.0,-95.0,-15.0,4.0,11.0,36005.0,149.0,10805003.0,-
4,11.0,-95.0,-16.0,2.9999999999999996,10.000000000000002,36367.0,136.0,10805003.0,-
...,...,...,...,...,...,...,...,...,...
1495,11.0,-95.99999999999999,-15.0,1.9999999999999993,11.0,195371.0,281.0,10805003.0,-
1496,11.0,-95.99999999999999,-16.0,8.000000000000002,8.0,0.0,0.0,10805003.0,76.0
1497,11.0,-95.99999999999999,-12.000000000000002,4.999999999999999,10.000000000000002,31222.000000000004,138.0,10805003.0,-
1498,11.0,-95.0,-12.000000000000002,1.0000000000000002,11.0,10180.0,125.0,10805003.0,-


## Encoded data

In [50]:
pd.DataFrame(encoded_data, 
             columns = ['Encode%s'%(number) for number in range(encoded_data.shape[1])])

,Encode0,Encode1,Encode2,Encode3,Encode4,Encode5,Encode6,Encode7
0,0.0,0.727273,0.2,0.454545,0.222222,0.204021,0.210700,0.0
1,0.0,0.636364,0.3,0.545455,0.222222,0.082674,0.119342,0.0
2,0.0,0.727273,0.7,0.636364,0.444444,0.021854,0.023045,0.0
3,0.0,0.818182,0.4,0.500000,0.555556,0.118949,0.122634,0.0
4,0.0,0.818182,0.3,0.454545,0.444444,0.120144,0.111934,0.0
...,...,...,...,...,...,...,...,...
1495,0.0,0.727273,0.4,0.409091,0.555556,0.645441,0.231276,0.0
1496,0.0,0.727273,0.3,0.681818,0.222222,0.000000,0.000000,0.0
1497,0.0,0.727273,0.7,0.545455,0.444444,0.103147,0.113580,0.0
1498,0.0,0.818182,0.7,0.363636,0.555556,0.033631,0.102881,0.0


## Decoded data

In [59]:
decoded_csv = getOriginalData(original_data = selected_data, 
                              categorical_data = original_data['PINGMAX'], 
                              decoded_data = decoded_data)
decoded_csv = pd.DataFrame(decoded_csv, 
             columns = ['CellID','RSRP','RSRQ','SNR','CQI','DL_bitrate',
                        'UL_bitrate','RAWCELLID', 'PINGMAX'])
decoded_csv

,CellID,RSRP,RSRQ,SNR,CQI,DL_bitrate,UL_bitrate,RAWCELLID,PINGMAX
0,11.300176,-96.625984,-12.972424,5.179847,10.325671,139779.08,347.85675,10805003.0,-
1,11.321503,-96.62204,-13.15651,4.933017,10.431916,141217.14,349.1485,10805003.0,-
2,11.323632,-96.18943,-13.252827,4.3936057,10.539391,139052.22,313.2072,10805003.0,-
3,11.300129,-96.47737,-13.099199,4.504618,10.230279,137602.17,320.0731,10805003.0,-
4,11.297507,-96.4423,-13.050088,4.5761824,10.321843,137835.8,328.4793,10805003.0,-
...,...,...,...,...,...,...,...,...,...
1495,11.283201,-96.99922,-13.088597,5.048996,9.846761,134506.4,312.12405,10805003.0,-
1496,11.30672,-96.261215,-13.116018,4.691628,10.528472,141032.8,317.56036,10805003.0,-
1497,11.322384,-96.30301,-13.195086,4.6002502,10.478329,138515.55,324.9767,10805003.0,-
1498,11.322447,-96.19646,-13.175442,4.2032623,10.569428,136575.69,337.78,10805003.0,-


In [72]:
np.unique(decoded_csv['PINGMAX'])

array(['-', '100.0', '182.0'], dtype=object)

In [73]:
np.unique(original_csv['PINGMAX'])

array(['-', '100.0', '109.0', '112.0', '114.0', '123.0', '128.0', '133.0',
       '182.0', '213.0', '278.0', '503.0', '74.0', '75.0', '76.0', '77.0',
       '78.0', '79.0', '80.0', '81.0', '82.0', '83.0', '84.0', '85.0',
       '87.0', '90.0', '93.0', '94.0'], dtype=object)